In [1]:
import pandas as pd
import torch
import lightning as L

from torch.utils.data import DataLoader

from model.modeling_demolta import DeMOLTaConfig
from trainer import LitMOLA, SaveTrainableParamsCheckpoint

In [2]:
torch.set_float32_matmul_precision('medium')

In [3]:
BATCH_SIZE = 4
MAX_STEP = 100000000
TEXT_MODEL_NAME = 'facebook/galactica-125m'

In [4]:
smiles_to_filter = pd.read_csv('./data/test.csv')['SMILES'].tolist()

In [5]:
train_dataloader = get_mola_dataloader(
    df_path='./preproc/pretrain.csv',
    ignore_smiles=smiles_to_filter,
    tokenizer_name=TEXT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

Using pad_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./preproc/pretrain.csv')

In [9]:
df.head().sort_values(by='smiles', key=lambda x: -x.str.len())

,smiles,query,answer
1,CC(=O)N1c2ccc(S(=O)(=O)N3CCCC3)cc2C[C@@H]1C(=O...,Give me the SMILES of this molecule.,CC(=O)N1c2ccc(S(=O)(=O)N3CCCC3)cc2C[C@@H]1C(=O...
4,CCOC(=O)c1sc(NC(=O)CCCS(=O)(=O)c2ccc(F)cc2)nc1...,Give me the SMILES string of this compound.,CCOC(=O)c1sc(NC(=O)CCCS(=O)(=O)c2ccc(F)cc2)nc1...
2,Cc1nc(-c2ccc(NC(=O)[C@H]3C[C@H]4CC[C@@H]3O4)cc...,What is the SMILES string of this compound?,Cc1nc(-c2ccc(NC(=O)[C@H]3C[C@H]4CC[C@@H]3O4)cc...
3,COc1ccc(C(=O)[C@H](C)Sc2nc(-c3cccs3)n[n-]2)cc1OC,What is the SMILES of this compound?,COc1ccc(C(=O)[C@H](C)Sc2nc(-c3cccs3)n[n-]2)cc1OC
0,O=C1NCCN1[C@@H]1CCC[NH+](Cc2cccc(O)c2)C1,Give me the SMILES of this molecule.,O=C1NCCN1[C@@H]1CCC[NH+](Cc2cccc(O)c2)C1


In [6]:
val_dataloader = get_mola_dataloader(
    df_path='./preproc/pretrain_val.csv',
    ignore_smiles=smiles_to_filter,
    tokenizer_name=TEXT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

Using pad_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [7]:
demolta_config = DeMOLTaConfig(
    num_layers=12,
    hidden_dim=384,
    ff_dim=1536,
    num_heads=6,
)

In [ ]:
service_account.Credentials.from_service_account_file('./auth/flowing-banner-391105-04efc2e014a8.json')

In [8]:
lit_model = LitMOLA(
    demolta_config=demolta_config,
    text_model_name=TEXT_MODEL_NAME,
)

In [9]:
checkpoint_callback = SaveTrainableParamsCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoint/',
    filename='mola-pretrain-{step}-{train_loss:.4f}-{val_loss:.4f}',
    save_top_k=3,
)

In [10]:
trainer = L.Trainer(
    accelerator='gpu',
    precision='bf16',
    max_steps=MAX_STEP,
    callbacks=[checkpoint_callback],
    gradient_clip_val=1.0,
    val_check_interval=10000,
    limit_val_batches=1000,
)


c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\fabric\connector.py:554: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(lit_model, train_dataloader, val_dataloader)

Missing logger folder: c:\Users\dust\Documents\Dacon\med\demolta\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params
--------------------------------
0 | model | MOLLA | 173 M 
--------------------------------
48.5 M    Trainable params
125 M     Non-trainable params
173 M     Total params
694.105   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
c:\Users\dust\Documents\Dacon\med\demolta\model\modeling_demolta.py:148: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  'position': torch.FloatTensor(atom_positions)


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]